## Feature Engineering and CV based Winners' Solutions

continued from eda_4_29

New in this notebook

To-do: 
- ~~number of active month~~
- ~~More features related to history products~~
- new product each month in history


Features:
- eda_4_28
    - average of products for each (customer, product) pair
    - exponent weighted average of products each (customer, product) pair
    - time since presence of products, distance to the first 1
    - time to the last positive flank (01)
    - time to the last negative flank (10)
    - time to the last 1, to the nearest product purchase
    - time to the first 1, to the first product purchase
- before eda_4_25
    - customer info in the second month
    - products in the first month
    - combination of first and second month `ind_actividad_cliente`
    - combination of first and second month `tiprel_1mes`
    - combination of first month product by using binary number (`target_combine`)
    - encoding `target_combine` with 
        - mean number of new products
        - mean number of customers with new products
        - mean number of customers with each new products
    - Count patterns in the last `max_lag` months
    - Number of month to the last time the customer purchase each product
        - CV@2015-12-28: mlogloss=1.29349
        - Private score: 0.0302475, public score: 0.0299266
- eda_4_25
    - Use all available history data
        - E.g., for 2016-05-28 train data, use all previous months, for 2015-02-28, use 1 lag month. 
        - Need to create test set that use the same amount of previous months for each training data set. 
        - This is from [the second winner's solution](https://www.kaggle.com/c/santander-product-recommendation/discussion/26824), his bold part in paragraph 4.
    - Combine models trained on 2016-05-28 and 2015-06-28:
        - Private score: 0.0304583, public score: 0.0300839
        - This is to catch both seasonality and trend, presented in 2015-06-28 and 2016-05-28, respectively. 
        - This idea is mentioned by many winners, like [11-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823) and [14-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26808)

- eda_4_27
    - put 2015-06-28 and 2016-05-28 in the same data set, with the same lag=5
        - Private score:0.0303096, public score: 0.0299867
        - Different as [11-th winner's discussion](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823)
            > We tested this by adding 50% of May-16 data to our June model and sure enough, we went from 0.0301 to 0.0303. Then, we built separate models for Jun and May, but the ensemble didn’t work. We weren’t surprised because June data is better for seasonal products, and May data is better for trend products. And vice-versa, June data is bad for trend products and May data is bad for seasonal products. So, they sort of cancelled each other out.

        - But my score is always worse than theirs, maybe this is the reason why we have different observations

# Draft

In [1]:
from santander_helper import *

In [2]:
history = count_history('2016-02-28', 10)

In [3]:
month1 = '2016-02-28'
max_lag=10
    
month_new = month_list.index(month1)+1
month_end = month_list.index(month1)
month_start = month_end-max_lag+1

# Check if month_new is the last month
if month_new<len(month_list)-1:
    # Customers with new products in month_new
    customer_product_pair = pd.read_hdf('../input/customer_product_pair.hdf', 'customer_product_pair')
    ncodpers_list = customer_product_pair.loc[customer_product_pair.fecha_dato==month_list[month_new], 
        'ncodpers'].unique().tolist()

# Load data for all the lag related months
df = []
for m in range(month_start, month_end+1):
    df.append(pd.read_hdf('../input/data_month_{}.hdf'.format(month_list[m]), 'data_month'))

# concatenate data
df = pd.concat(df, ignore_index=True)

# Drop irrelevant features, which never occur in models
df = df.loc[:, ['fecha_dato']+cat_cols+target_cols]
# If month_new is not the last month, keep only customers with new products 
if month_new<len(month_list)-1:
    # select customers if this is not test set
    df = df.loc[df.ncodpers.isin(ncodpers_list), :]

# set ncodpers and fecha_dato as index
df.set_index(['ncodpers', 'fecha_dato'], inplace=True)

# unstack to make month as columns, now we have MultiIndex with months and 
# products in two levels
df = df.unstack(level=-1, fill_value=np.nan)

In [60]:
lags = df[target_cols].copy()
level1 = [month_list.index(month1)+1 - month_list.index(k) for k in lags.columns.get_level_values(1)]
level0 = lags.columns.get_level_values(0).tolist()
lags.columns = [l0+'_lag_'+str(l1) for l0, l1 in zip(level0, level1)]

In [55]:
dt.head()

,ind_cco_fin_ult1_lag_10,ind_cco_fin_ult1_lag_9,ind_cco_fin_ult1_lag_8,ind_cco_fin_ult1_lag_7,ind_cco_fin_ult1_lag_6,ind_cco_fin_ult1_lag_5,ind_cco_fin_ult1_lag_4,ind_cco_fin_ult1_lag_3,ind_cco_fin_ult1_lag_2,ind_cco_fin_ult1_lag_1,...,ind_valo_fin_ult1_lag_10,ind_valo_fin_ult1_lag_9,ind_valo_fin_ult1_lag_8,ind_valo_fin_ult1_lag_7,ind_valo_fin_ult1_lag_6,ind_valo_fin_ult1_lag_5,ind_valo_fin_ult1_lag_4,ind_valo_fin_ult1_lag_3,ind_valo_fin_ult1_lag_2,ind_valo_fin_ult1_lag_1
ncodpers,,,,,,,,,,,,,,,,,,,,,
15889,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
15896,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
15925,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
x_train, y_train, weight_train = create_train('2016-03-28', max_lag=10, pattern_flag=True)

In [51]:
target = []
for m1, m2 in tqdm.tqdm_notebook(list(zip(month_list[:-2], month_list[1:-1]))):
    df1 = pd.read_hdf('../input/data_month_{}.hdf'.format(m1)).loc[:, ['ncodpers']+target_cols]
    df2 = pd.read_hdf('../input/data_month_{}.hdf'.format(m2)).loc[:, ['ncodpers']+target_cols]

    df1.set_index('ncodpers', inplace=True)
    df2.set_index('ncodpers', inplace=True)

    dt = df2.subtract(df1).fillna(0.0)
    dt.reset_index(inplace=True)
    dt = dt.melt(id_vars='ncodpers')
    dt['variable'] = dt['variable'].map({k: i for i, k in enumerate(target_cols)})
    dt = dt.loc[dt['value']==1.0]
    dt.drop('value', axis=1, inplace=True)
    dt.reset_index(drop=True, inplace=True)
    dt['fecha_dato'] = m2
    
    target.append(dt)
    
target = pd.concat(target, ignore_index=True)

In [50]:
target.tail

,ncodpers,variable
558318,1542723,18
558319,1543771,18
558320,1544736,18
558321,1545426,18
558322,1547241,18


In [46]:
def all_target():
    target = []
    for m1, m2 in tqdm.tqdm_notebook(list(zip(month_list[:-2], month_list[1:-1]))):
        df1 = pd.read_hdf('../input/data_month_{}.hdf'.format(m1)).loc[:, ['ncodpers']+target_cols]
        df2 = pd.read_hdf('../input/data_month_{}.hdf'.format(m2)).loc[:, ['ncodpers']+target_cols]

        df1.set_index('ncodpers', inplace=True)
        df2.set_index('ncodpers', inplace=True)

        dt = df2.subtract(df1).fillna(0.0)
        dt.reset_index(inplace=True)
        dt = dt.melt(id_vars='ncodpers')
        dt['variable'] = dt['variable'].map({k: i for i, k in enumerate(target_cols)})
        dt = dt.loc[dt['value']==1.0]
        dt.drop('value', axis=1, inplace=True)
        dt.reset_index(drop=True, inplace=True)

        target.append(dt)

    target = pd.concat(target, ignore_index=True)
    
    return target

,ncodpers,variable
0,16162,0
1,18280,0
2,19878,0
3,19990,0
4,20234,0
5,20981,0
6,21876,0
7,23334,0
8,23551,0
9,27533,0


In [15]:
targets = []
for m1, m2 in tqdm.tqdm_notebook(list(zip(month_list[:-2], month_list[1:-1]))):
    target1 = obtain_target(m2)
    target1['fecha_dato'] = m2
    targets.append(target1)
    
targets = pd.concat(targets, ignore_index=True, copy=False)
targets.to_hdf('../input/targets.hdf', 'targets')

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: ind_hip_fin_ult1_lag_10, dtype: float64